In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import unicodedata
import os
import webbrowser
import html5lib
from openpyxl import workbook
from datetime import datetime
import requests

In [21]:
path = '../Master_tables_GBDC_Investment.xlsx'
dataframes = pd.read_excel(path, sheet_name=None)

In [102]:
process_tables = {}
process_tables_shape = []
if not os.path.exists('../PT_csv_file'):
    os.makedirs('../PT_csv_file')


def run_process_function(dataframes, process_tables, process_tables_shape):
    path = '../process_tables_GBDC_Investment.xlsx'
    writer = pd.ExcelWriter(path=path, engine='openpyxl')
    for dataframe in dataframes:
        print(dataframe)
        processed_table = process_table_function(dataframes[dataframe])
        process_tables[dataframe] = processed_table
        process_tables_shape = processed_table.shape
        processed_table.to_excel(
            writer, sheet_name=dataframe.replace(',', ''), index=False)
        processed_table.to_csv(
            '../PT_csv_file/'+dataframe.replace(',', '')+'.csv')
        writer.book.save(path)
    writer.close()

In [115]:
def shape(count, df):
    print(f"{count} : shape : {df.shape}")
    count += 1
    return count


def dropna_col_row(df):
    df = df.dropna(how='all', axis=0).reset_index(drop=True)
    df = df.dropna(how='all', axis=1).reset_index(drop=True)
    return df


def drop_if_contain(pattern, df):
    matching_rows = df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    df = df[~matching_rows]
    return df

In [119]:
def process_table_function(soi_table_df):
    count = 1
    count = shape(count, soi_table_df)
    soi_table_df = soi_table_df.replace(
        r'^\s*\$\s*$', '', regex=True).replace(r'\n', '', regex=True)
    soi_table_df = soi_table_df.replace('-', '0')
    soi_table_df = dropna_col_row(soi_table_df)
    soi_table_df = soi_table_df.apply(
        lambda x: x.strip() if isinstance(x, str) else x)
    count = shape(count, soi_table_df)

    # drops all the extra top row
    pattern = r'Net asset value per common share|How We Addressed the Matter in Our Audit'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax(
        )+1:].reset_index(drop=True)
    count = shape(count, soi_table_df)

    # drops all the extra bottom row
    pattern = r'Total\s+Investments'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row
    if soi_table_df[matching_rows].index[0] < 20:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
            drop=True)
    else:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
            drop=True)
    count = shape(count, soi_table_df)

    # drop all the col name
    pattern = r'(?:Spread\s*Above|cost|Percentage|Above)'
    soi_table_df = drop_if_contain(pattern, soi_table_df)
    count = shape(count, soi_table_df)
    soi_table_df = dropna_col_row(soi_table_df)
    count = shape(count, soi_table_df)

#rename col
    num_cols = soi_table_df.shape[1]
    data_col_mapper = dict(zip(soi_table_df.columns.to_list(), [
        i for i in range(0, num_cols)]))
    soi_table_df = soi_table_df.rename(columns=data_col_mapper)

    # if row value is "no value" and get col value is not nan then replace the "no value" with the not nan value
    for index, row in soi_table_df.iterrows():
        for column in soi_table_df.columns:
            if row[column] == "No Value":
                next_column = soi_table_df.soi_table_df.columns.get_loc(column) + 1


    return soi_table_df


run_process_function(dataframes=dataframes, process_tables=process_tables,
                     process_tables_shape=process_tables_shape)

March 31 2013
1 : shape : (424, 31)
2 : shape : (424, 15)
3 : shape : (424, 15)
4 : shape : (424, 15)
5 : shape : (404, 15)
6 : shape : (404, 10)
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value
No Value

KeyboardInterrupt: 

[]